# Assignment Week 3 Capstone Data Science

### First stage is to scrape the wiki page for the data. 

#### Import urllib library which is used to open URLs. 

In [1]:
import requests

In [2]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#### Import the Beautiful Soup library which is used to parse the HTML.

In [3]:
from bs4 import BeautifulSoup

#### Parse the HTML of the URL into a Beautiful Soup tree format

In [4]:
soup = BeautifulSoup(website_url,'lxml')

In [5]:
#Find the title
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

#### Look through and find the required table. Data required starts with HTML tag "table"

In [6]:
#print(soup.prettify())

#### Bring back the table using find

In [7]:
pcode_table=soup.find('table', class_='wikitable sortable')
#pcode_table

#### Each row in the table is starts with "tr" and the data within is row is within "td". 

##### Loop through the pcode table and find the "tr" then the "td" tags. If the row has 3 columns then store the data in lists A (first column), B (second column) and C(third column).

In [8]:
A=[]
B=[]
C=[]

for row in pcode_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        

In [9]:
#Have a quick check of the first element of the each list.
A[0]

'M1A'

In [10]:
B[0]

'Not assigned'

In [11]:
C[0]

'Not assigned\n'

#### There appears to be some newlines in the columns. Use rstrip to go through list and clean.

In [12]:
j=0
for i in A:
    A[j]=i.rstrip()
    j=j+1
j=0
for i in B:
    B[j]=i.rstrip()
    j=j+1
j=0
for i in C:
    C[j]=i.rstrip()
    j=j+1

#### Import the pandas library and create a dataframe df with columns PostalCode, Borough and Neighbourhood and assign the column lists to the dataframe.

In [13]:
import pandas as pd
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighbourhood']=C

#### Check the top and bottom of the dataframe to ensure it's captured

In [14]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [15]:
df.tail()

,PostalCode,Borough,Neighbourhood
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor
287,M9Z,Not assigned,Not assigned


#### Need to ensure that rows are removed that don't have a borough or neighbourhood assigned

In [16]:
df = df[df['Borough'] != 'Not assigned']

In [17]:
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


#### Combine neighbourhoods (comma seperated), and remove duplicate rows of Postalcodes and Boroughs.

In [18]:
dfnew = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(','.join)

In [19]:
dfnew.head()

PostalCode  Borough    
M1B         Scarborough                           Rouge,Malvern
M1C         Scarborough    Highland Creek,Rouge Hill,Port Union
M1E         Scarborough         Guildwood,Morningside,West Hill
M1G         Scarborough                                  Woburn
M1H         Scarborough                               Cedarbrae
Name: Neighbourhood, dtype: object

#### Convert back to a dataframe and reset the index

In [20]:
dfnew2 = pd.DataFrame(dfnew)
dfnew2 = dfnew2.reset_index()

In [21]:
dfnew2.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Change Neighbour to Borough if it's unassigned.

In [22]:
dfnew2.loc[(dfnew2['Neighbourhood'] == 'Not assigned'),'Neighbourhood']=dfnew2['Borough']

#### Shape of dataframe

In [23]:
dfnew2.shape

(103, 3)